## **6. Securing Federated Learning**

### **Federated Learning with a Trusted Aggregator**

In [0]:
!pip install syft

     |████████████████████████████████| 215kB 6.8MB/s 
     |████████████████████████████████| 204kB 49.1MB/s 
     |████████████████████████████████| 71kB 26.0MB/s 
     |████████████████████████████████| 450kB 48.3MB/s 
     |████████████████████████████████| 1.4MB 43.9MB/s 
     |████████████████████████████████| 256kB 50.1MB/s 
     |████████████████████████████████| 389kB 41.5MB/s 
     |████████████████████████████████| 276kB 45.2MB/s 
     |████████████████████████████████| 51kB 20.5MB/s 
     |████████████████████████████████| 122kB 46.0MB/s 
  Stored in directory: /root/.cache/pip/wheels/ad/9a/f4/3105b5209674ac77fcca7fede95184c62a95df0196888e0e76
  Stored in directory: /root/.cache/pip/wheels/16/27/a1/775c62ddea7bfa62324fd1f65847ed31c55dadb6051481ba3f
Successfully built zstd pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Found existing installation: msgpack 0.5.6
    Uninstalling msgpack-0.5.6:
     

In [0]:
import syft as sy
import torch as th
hook = sy.TorchHook(th)
from torch import nn, optim

W0627 13:54:31.128722 140423213606784 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0-rc1.so'
W0627 13:54:31.142996 140423213606784 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")

In [0]:
bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([bob, alice])

W0627 13:54:32.720081 140423213606784 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0627 13:54:32.721551 140423213606784 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0627 13:54:32.722666 140423213606784 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0627 13:54:32.723506 140423213606784 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0627 13:54:32.724455 140423213606784 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0627 13:54:32.725634 140423213606784 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior


<VirtualWorker id:secure_worker #objects:0>

In [0]:
# A toy dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad = True)
target = th.tensor([[1.],[1],[0],[0]], requires_grad=True)


In [0]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

data_alice = data[2:].send(alice)
target_alice = target[2:].send(alice)

In [0]:
model = nn.Linear(2,1)

Now, we''ll have two different model that'll be sended to both workers.

In [0]:
bobs_model = model.copy().send(bob)
alices_model = model.copy().send(alice)

In [0]:
bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.1)
alices_opt = optim.SGD(params=alices_model.parameters(),lr=0.1)

In [0]:
for i in range(10):

  # bob's model
  bobs_opt.zero_grad()
  bobs_pred = bobs_model(data_bob)
  bobs_loss = ((bobs_pred - target_bob)**2).sum()
  bobs_loss.backward()

  bobs_opt.step()
  bobs_loss = bobs_loss.get().data
  bobs_loss

  # alice's model
  alices_opt.zero_grad()
  alices_pred = alices_model(data_alice)
  alices_loss = ((alices_pred - target_alice)**2).sum()
  alices_loss.backward()

  alices_opt.step()
  alices_loss = alices_loss.get().data
  alices_loss

Now that we've trained both models separately, it's time to average them. For this, we'll send them to secure worker. 

In [0]:
alices_model.move(secure_worker)
bobs_model.move(secure_worker)

In [0]:
# here you should do model.weight.data.set_() but I keep having an error
model.weight.data = (((alices_model.weight.data + bobs_model.weight.data)/2).get())
model.bias.data = (((alices_model.bias.data + bobs_model.bias.data)/2).get())

### **Additive Secret Sharing**

Even this way (the one we did in the previous section), we need to trust a third party to see our raw data. 

We can modify this, by adding encription to our data. For that reason, we're going to use Additive Secret Sharing. We'll need the following functions.

In [0]:
import random

In [0]:
Q = 23740629843760239486723

In [0]:
def encrypt(x, n_shares=3):
  
  shares = list()
  
  for i in range(n_shares-1):
    shares.append(random.randint(0,Q))
    
  final_share = Q -(sum(shares)%Q) + x
  
  shares.append(final_share)
  
  return tuple(shares)

In [0]:
encrypt(5,3)

(14240640204254952100607, 22783298396949810508307, 10457321086315716364537)

In [0]:
def decrypt(shares):
  return (sum(shares)%Q)
  

In [0]:
decrypt(encrypt(4))

4

In [0]:
def add(a,b):
  
  c = list()
  
  assert(len(a) == len(b))
  
  for i in range(len(a)):
    c.append((a[i]+b[i])%Q)
    
  return tuple(c)

In [0]:
decrypt(add(encrypt(7),encrypt(7)))

14

We can also create other functions as subtraciont and multiplication

In [0]:
def subtract(a,b):
  
  c = list()
  
  assert(len(a) == len(b))
  
  for i in range(len(a)):
    c.append((a[i]-b[i])%Q)
    
  return tuple(c)

In [0]:
decrypt(subtract(encrypt(7),encrypt(2)))

5

In [0]:
# multiply encrypted umber by a known scalar
def multiply(a,scalar):
  
  c = list()
  
  for i in range(len(a)):
    c.append((a[i]*scalar)%Q)
    
  return tuple(c)

In [0]:
decrypt(multiply(encrypt(7),2))

14

### **Fixed Precision Encoding**

Our goal is to encode the weights by using the previous technique. However it only works with integer. FPE lets us do computation over decimal numbers using integers.

In [0]:
BASE = 10
PRECISION = 4
Q = 23740629843760239486723

In [0]:
def encode(x_dec):
  return int(x_dec*(BASE**PRECISION))% Q

In [0]:
def decode(x_fp):
  return (x_fp if x_fp <= Q/2 else x_fp-Q) / BASE**PRECISION

In [0]:
decode(encode(0.5))

0.5

### **Fixed Precision in PySyft**

In [0]:
bob = bob.clear_objects()
alice = alice.clear_objects()
secure_worker = secure_worker.clear_objects()

In [0]:
x = th.tensor([1,2,3,4,5])

In [0]:
x = x.share(bob,alice,secure_worker)
x

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:16564879343 -> bob:8196139741]
	-> (Wrapper)>[PointerTensor | me:68102753621 -> alice:83916737827]
	-> (Wrapper)>[PointerTensor | me:92935346880 -> secure_worker:59895894078]
	*crypto provider: me*

In [0]:
bob._objects

{8196139741: tensor([3180144277271009586, 3095836909958152134, 3918868126591434552,
         2121681609746945176, 2386369850837645924])}

In [0]:
y = x + x

In [0]:
y.get()

tensor([ 2,  4,  6,  8, 10])

In [0]:
x = th.tensor([0.1,0.2,0.3,0.4,0.5])
x

tensor([0.1000, 0.2000, 0.3000, 0.4000, 0.5000])

In [0]:
x = x.fix_prec()
x

(Wrapper)>FixedPrecisionTensor>tensor([100, 200, 300, 400, 500])

In [0]:
x = x.float_prec()
x

tensor([0.1000, 0.2000, 0.3000, 0.4000, 0.5000])